# Kennissessie Neurale Netwerken

# Deel 1
Deze sessie gaan we neurale netwerken trainen om simpele images te classificeren van de CIFAR-10 dataset. 

**Stap 1: data inladen**

De data bestaat uit 3 delen, waaronder de standaard train en testset, maar ook een validatieset die tijdens het trainen van het netwerk al een idee geeft hoe het netwerk op ongeziene data gaat performen.

Let op: onderstaande stuk code is totaal niet belangrijk om te begrijpen voor de rest van de notebook. Het zorgt er alleen voor dat de data gedownload wordt en in de juiste mapjes wordt gezet. Besteed niet teveel tijd aan het precies begrijpen wat er gebeurt!

In [1]:
import numpy as np
import os
import tarfile
from urllib.request import urlretrieve
import pickle
import random


def load_data():
    # training set, batches 1-4
    if not os.path.exists(os.path.join(os.getcwd(), "data")):
        os.makedirs(os.path.join(os.getcwd(), "data"))

        
    dataset_dir = os.path.join(os.getcwd(), "data")
    
    if not os.path.exists(os.path.join(dataset_dir, "cifar-10-batches-py")):
        print("Downloading data...")
        urlretrieve("http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar = tarfile.open(os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar.extractall(dataset_dir)
        tar.close()
        
    train_X = np.zeros((40000, 3, 32, 32), dtype="float32")
    train_y = np.zeros((40000, 1), dtype="ubyte").flatten()
    n_samples = 10000  # aantal samples per batch
    dataset_dir = os.path.join(dataset_dir,"cifar-10-batches-py")
    for i in range(0,4):
        f = open(os.path.join(dataset_dir, "data_batch_"+str(i+1)), "rb")
        cifar_batch = pickle.load(f,encoding="latin1")
        f.close()
        train_X[i*n_samples:(i+1)*n_samples] = (cifar_batch['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
        train_y[i*n_samples:(i+1)*n_samples] = np.array(cifar_batch['labels'], dtype='ubyte')

    # validation set, batch 5
    f = open(os.path.join(dataset_dir, "data_batch_5"), "rb")
    cifar_batch_5 = pickle.load(f,encoding="latin1")
    f.close()
    val_X = (cifar_batch_5['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    val_y = np.array(cifar_batch_5['labels'], dtype='ubyte')

    # labels
    f = open(os.path.join(dataset_dir, "batches.meta"), "rb")
    cifar_dict = pickle.load(f,encoding="latin1")
    label_to_names = {k:v for k, v in zip(range(10), cifar_dict['label_names'])}
    f.close()

    # test set
    f = open(os.path.join(dataset_dir, "test_batch"), "rb")
    cifar_test = pickle.load(f,encoding="latin1")
    f.close()
    test_X = (cifar_test['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    test_y = np.array(cifar_test['labels'], dtype='ubyte')


    print("training set size: data = {}, labels = {}".format(train_X.shape, train_y.shape))
    print("validation set size: data = {}, labels = {}".format(val_X.shape, val_y.shape))
    print("test set size: data = {}, labels = {}".format(test_X.shape, test_y.shape))

    return train_X, train_y, val_X, val_y, test_X, test_y, label_to_names


### Shuffle en reshape data
Bij veel datasets zit de data gegroepeerd waardoor het goed is om de trainingset te shufflen voordat het als input voor het neurale netwerk gebruikt wordt. Bij CIFAR10 is dit niet essentieel, maar we doen het toch even. 
De data moet daarna reshaped worden naar (nr_images, nr_channels\*image_size\*image_size) om correcte input voor onze MLP te leveren. Als we met CNN's aan de gang gaan is dit niet meer nodig.

In [2]:
nr_channels = 3
image_size = 32
nr_classes = 10
epochs = 20


# The data, shuffled and split between train and test sets:
train_X, train_y, val_X, val_y, test_X, test_y, label_to_names = load_data()

#Shuffle trainingset
train_set = list(zip(train_X, train_y))
random.shuffle(train_set)
train_X, train_y = zip(*train_set)
train_X = np.array(train_X).reshape(40000,nr_channels*image_size*image_size)
train_y = np.array(train_y)
val_X = val_X.reshape(10000,nr_channels*image_size*image_size)
test_X = test_X.reshape(10000,nr_channels*image_size*image_size)



training set size: data = (40000, 3, 32, 32), labels = (40000,)
validation set size: data = (10000, 3, 32, 32), labels = (10000,)
test set size: data = (10000, 3, 32, 32), labels = (10000,)


### Preprocessing
Bij CIFAR10 is er niet veel preprocessing nodig. Normalisatie van de data is vaak een goed idee, vantevoren berekenen we de gemiddelde pixelwaarde en bij het batchgewijs trainen normaliseren we de data aan de hand van die waarde. Bij grotere datasets die niet volledig in je geheugen passen moet je hier vaak wat slimmers toepassen zoals een volledige pass door je data waarbij je de mean en variance streaming berekend met bijv. met Welford's algorithm. Tegenwoordig wordt ook vaak geen standaardisatie meer toegepast maar gewoon een simpele division door de gemiddelde pixelwaarde.

In [3]:
def calc_mean_std(X):
    mean = np.mean(X)
    std = np.std(X)
    return mean, std

def normalize(data, mean, std):
    return (data-mean)/std

#De data van train_X is genoeg om de mean en std van de hele set nauwkeurig te benaderen
mean,std = calc_mean_std(train_X)
#train_y = train_y.reshape((-1, 1))
test_X = normalize(test_X,mean,std)
val_X = normalize(val_X,mean,std)
train_X = normalize(train_X ,mean,std)


### Define Network
Het is nu tijd om ons netwerk te definieren. We beginnen met een heel simpel MLP, en zullen dit later uitbreiden tot een mooi CNN. We gebruiken hierbij de "functional API" van Keras, in plaats van de Sequential API. Met Sequential kun je namelijk alleen maar sequentiele modellen definieren, en geen vertakkingen. Met de functional API kun je alles wat je maar wil, en de functional API is ook wat logischer in gebruik.

In [4]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten


def mlp():
    # Bij de eerste layer moeten we altijd input dimensions meegeven omdat TensorFlow en Theano STATIC computation graphs maken
    # Bij iets als PyTorch is dat niet nodig, omdat PyTorch DYNAMIC computation graphs maakt
    input = Input(shape=(nr_channels*image_size*image_size,))
    
    # Eerste dense layer, met een ReLU activatiefunctie
    input_layer = Dense(units=100, activation='relu')(input)
    hidden_layer = Dense(units=100, activation='relu')(input_layer)
    
    # Output layer heeft een softmax activatiefunctie in plaats van een ReLU.
    # Softmax zorgt ervoor dat alle outputs optellen tot 1 en exponentieel genormaliseerd worden
    # Zie de wikipedia pagina van softmax voor meer info
    output_layer = Dense(units=nr_classes, activation='softmax')(hidden_layer)
    
    model = Model(inputs=input, outputs=output_layer)
    
    # Het model moet nog gecompiled worden en loss+learning functie gespecificeerd worden
    # Als loss function gebruiken we sparse categorical crossentropy. Dat is log loss met sparse coding.
    # Sparse coding betekent hier dat we geen one-hot encoding hebben toegepast op de labels,
    # maar dat het direct integer category labels zijn.
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def mlp_2():
    pass

Using TensorFlow backend.


### Minibatches 
Om het trainen wat vlotter te laten lopen gebruiken trainen we het netwerk per batch in plaats van per plaatje. De gradients worden dan berekend per batch in plaats van de hele set, waardoor het significant sneller traint vanwege parallelisatie-optimalisaties. Ook wordt de gradient gemiddeld over meerdere samples, wat een noisy gradient voorkomt. Minibatch training kan standaard in Keras bij het fitten van het model.

### Pipeline
Alle puzzelstukjes staan nu op zijn plek, dus kunnen we via een pipeline alles aan elkaar gaan linken en het model trainen en evalueren. 

In [ ]:
from sklearn.metrics import classification_report

model = mlp()

# model.summary() geeft een prachtig overzicht van je netwerk, inclusief in welke laag hoeveel parameters zitten
model.summary()

model.fit(x=train_X, y=train_y, batch_size=50, epochs=10, validation_data=(val_X,val_y), verbose=2)
predictions = np.array(model.predict(test_X, batch_size=100))
test_y = np.array(test_y, dtype=np.int32)
# Pak de hoogste prediction
predictions = np.argmax(predictions,axis=1)

# Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

### Opdracht
Ons netwerk is nu erg simpel en geeft matige resultaten. Er zijn veel manieren om het netwerk beter te maken. In deze opdracht gaan we daarmee aan de slag. Maak veel gebruik van de [Keras documentatie](https://keras.io/)!

* **Extra Layers**  bijvoorbeeld het toevoegen van meer layers. Pas model mlp() aan of maak een nieuw model in mlp_2() die meer layers heeft en kijk of dit betere resultaten oplevert. Het is verstandig om niet te vaak je model te evalueren op de testset, maar puur te kijken naar de training en validatie error of accuracy. Anders is het gevaar dat het netwerk overfit op de testset.

* **Meer Units** Wat ook goed werkt is in plaats van meer layers toe te voegen, de bestaande layers groter maken. Dit kan door het verhogen van het aantal units per layer, waardoor er meer informatie per layer beschikbaar is. Experimenteer hiermee.

* **Weight Initialisatie** De weights van de layers worden nu geïnitialiseerd vanuit de normaalverdeling. Kijk bij de [documentatie](https://keras.io/initializers/) voor meer mogelijkheden

* **Learning Function en Learning Rate** Het netwerk heeft nu als optimizer ADAM, het kan best zijn dat voor dit probleem er een betere oplossing is. [Hier](https://keras.io/optimizers/) wordt uitgelegd welke standaard optimizers er zijn, maar ook hoe een custom optimizer gemaakt kan worden. Maak een custom optimizer voor SGD met learning rate 0.1 en een passende learning rate decay erbij. Voeg dit toe aan je netwerk.

* **Externe parameters** Ons netwerk runt nu maar een paar epochs, door dit te verhogen kan het netwerk beter convergeren. Pas echter op voor overfitting! Hier komen we in deel 2 op terug.


# Deel 2: Convolutional Neural Networks
De stap om van een MLP naar een CNN te gaan is niet zo groot, in plaats van een geflattened image gebruiken we nu de originele images als input, en passen we ons model aan om convolutie en pooling layers toe te voegen.

In [ ]:
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from sklearn.metrics import classification_report

# Laad en normaliseer de data (opnieuw)
train_X, train_y, val_X, val_y, test_X, test_y, label_to_names = load_data()

# Conv nets trainen duurt erg lang op CPU, dus we gebruiken maar een klein deel
# van de data nu, als er tijd over is kan je proberen je netwerk op de volledige set te runnen
train_X = train_X[:10000]
train_y = train_y[:10000]

def calc_mean_std(X):
    mean = np.mean(X)
    std = np.std(X)
    return mean, std

def normalize(data, mean, std):
    return (data-mean)/std

#De data van train_X is genoeg om de mean en std van de hele set nauwkeurig te benaderen
mean,std = calc_mean_std(train_X)
test_X = normalize(test_X,mean,std)
val_X = normalize(val_X,mean,std)
train_X = normalize(train_X ,mean,std)

In [ ]:
def conv_net():
    # We definieren de input van het netwerk als de shape van de input,
    # minus de dimensie van het aantal plaatjes, uiteindelijk dus (3, 32, 32).
    input = Input(shape=train_X.shape[1:])
    
    # Eerste convolutielaag
    # Padding valid betekent dat we enkel volledige convoluties gebruiken, zonder padding
    # Data format channels_first betekent dat de channels eerst komen, en dan pas de size van ons plaatje
    # Dus (3, 32, 32) in plaats van (32, 32, 3)
    conv = Conv2D(filters=16, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu')(input)
    
    # Nog een convolutielaag, dit keer met stride=2 om de inputsize te verkleinen
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu', strides=(2, 2))(conv)
    
    #Voeg een flatten laag toe, om te schakelen naar de dense layer
    flatten = Flatten()(conv)
   
    # De softmax laag voor de probabilities 
    output_layer = Dense(units=nr_classes, activation='softmax')(flatten)
    
    model = Model(inputs=input, outputs=output_layer)
    
    # Het model moet nog gecompiled worden en loss+learning functie gespecificeerd worden
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


model = conv_net()

model.fit(x=train_X, y=train_y, batch_size=50, epochs=10, validation_data=(val_X, val_y), verbose=2)
predictions = np.array(model.predict(test_X, batch_size=100))
test_y = np.array(test_y, dtype=np.int32)
#Pak de hoogste prediction
predictions = np.argmax(predictions, axis=1)

#Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

### Opdracht
Maak ook hier ons netwerk beter door bijvoorbeeld:

* **Extra Layers**  Voeg net als bij de MLP extra layers toe, en experimenteer met de parameters. Kijk vooral ook naar dropout layers, aangezien het netwerk nu enorm aan het overfitten is kan dit een hele goede toevoeging zijn. Probeer ook pooling layers te gebruiken in plaats van een grotere stride voor downsampling van je netwerk. Idealiter wil je zoveel mogelijk lagen toevoegen en downsamplen tot je niet meer kan vanwege dimensie problemen.

* **Meer filters of grotere filter size** Vaak beginnen netwerken met weinig filters en wordt het uitgebreid naar bijvoorbeeld 256 filters per convolutielaag. Dit zorgt wel voor langere runtimes. De filter size wordt tegenwoordig vaak op 3 gehouden, maar het kan in de eerste paar lagen effectief zijn om dit wat groter te maken, bijvoorbeeld 5 of 7.

* **Andere parameters** Bedenk wat er nog meer verbeterd kan worden, en experimenteer hiermee!
    
    